In [9]:
import pandas as pd
import scipy.io as sio
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

def Array_To_DataFrame(matDF, varName) :
    npArray = np.array([])
    for i in range (matDF[varName].shape[0]) :
        npArray = np.concatenate((npArray, matDF[varName][i].flatten()))
    return npArray

def matFile_To_DF(matFile) :
    tempDF = sio.loadmat(matFile)
    matDataMidFiltered = {dictKey:dictValue for dictKey, dictValue in tempDF.items() if dictKey[0] != '_'}
    matMidDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMidFiltered.items()})
    return matMidDF

#import downriver bottom data
matDataMini2 = sio.loadmat('2012-13\\hobo_minilander_3826_Dep2.mat')
matDataMini3 = sio.loadmat('2012-13\\hobo_minilander_3829_Dep3.mat')

#mat lab date origin
origin = np.datetime64('0000-01-01', 'D') - np.timedelta64(1, 'D')

matDataMini2Filtered = {dictKey:dictValue for dictKey, dictValue in matDataMini2.items() if dictKey[0] != '_'}
matMini2DF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMini2Filtered.items()})
matDataMini3Filtered = {dictKey:dictValue for dictKey, dictValue in matDataMini3.items() if dictKey[0] != '_'}
matMini3DF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMini3Filtered.items()})

matMiniDF = pd.concat([matMini2DF, matMini3DF])
tmsMiniriver = matMiniDF['DN'] * np.timedelta64(1, 'D') + origin

fileCollection = ['2012-13\\hobo_minilander_3826_Dep2.mat', '2012-13\\hobo_minilander_3829_Dep3.mat']
preTimeStamp = None
postTimeStamp = None
for timeFile in fileCollection: 
    countDF = matFile_To_DF(timeFile)
    DNArray = Array_To_DataFrame(countDF, 'DN')
    if pd.isnull(preTimeStamp):
        preTimeStamp = DNArray.min() 
    else: 
        if preTimeStamp > DNArray.min():
            preTimeStamp = DNArray.min()
    if pd.isnull(postTimeStamp):
        postTimeStamp = DNArray.max()
    else: 
        if postTimeStamp < DNArray.max():
            postTimeStamp = DNArray.max()

#reading in flow data
flowDF = pd.read_csv('2012_2013alsea.csv')
flowDF['fRate'] = flowDF['max_va'] * 0.0283168
#create a new column for flow dates
for index, row in flowDF.iterrows():
    calcDate = pd.Timestamp(year=int(row['begin_yr']), month=int(row['month_nu']), day=int(row['day_nu']))
    flowDF.loc[index, 'DN'] = calcDate

tmsFlow = flowDF['DN']
tempFigLabels = ['Miniriver']
tempfig = px.line(x = tmsMiniriver, y = [matMiniDF['T']], color_discrete_sequence= ["purple"], title = "Mini")
for idx in range(len(tempFigLabels)):
    tempfig.data[idx].name = tempFigLabels[idx]
    tempfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    tempfig.data[idx].legendgroup = tempFigLabels[idx]
salfig = px.line(x= tmsMiniriver, y = [matMiniDF['S']], color_discrete_sequence = ["purple"], title = "Mini")
for idx in range(len(tempFigLabels)):
    salfig.data[idx].name = tempFigLabels[idx]
    salfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    salfig.data[idx].legendgroup = tempFigLabels[idx]
flowFigLabel = ['Flow Rate']
flowfig = px.line(x = tmsFlow, y = [flowDF['fRate']], color_discrete_sequence= ["green"], title = "Flow Rate")
for idx in range(len(flowFigLabel)):
    flowfig.data[idx].name = flowFigLabel[idx]
    flowfig.data[idx].hovertemplate = 'variable=' + flowFigLabel[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    flowfig.data[idx].legendgroup = flowFigLabel[idx]

tempfig.update_layout(title=dict(text= "Bottom Temperature for 2012-2013", font=dict(size=25)))
tempfig.update_xaxes(tickangle=30)
tempfig.update_xaxes(rangeslider_visible=True)
tempfig.update_xaxes(range = [pd.Timestamp(preTimeStamp * np.timedelta64(1, 'D') + origin),pd.Timestamp('2013-05-25')])

salfig.update_layout(title=dict(text= "Bottom Salinity for 2012-2013", font=dict(size=25)))
salfig.update_xaxes(tickangle=30)
salfig.update_xaxes(rangeslider_visible=True)
salfig.update_xaxes(range = [pd.Timestamp(preTimeStamp * np.timedelta64(1, 'D') + origin),pd.Timestamp('2013-05-25')])

flowfig.update_layout(title=dict(text= "Flow Rate for 2012-2013", font=dict(size=25)))
flowfig.update_xaxes(tickangle=30)
flowfig.update_xaxes(rangeslider_visible=True)
flowfig.update_xaxes(range = [pd.Timestamp(preTimeStamp * np.timedelta64(1, 'D') + origin),pd.Timestamp('2013-05-25')])

tempfig.update_layout(xaxis_title="Date", yaxis_title="Temperature (°C)", legend_title="Locations")
salfig.update_layout(xaxis_title="Date", yaxis_title="Salinity", legend_title="Locations")
flowfig.update_layout(xaxis_title="Date", yaxis_title="Cubic Meters per Second",legend_title="Locations")

tempfig.show()
salfig.show()
flowfig.show()

